In [ ]:
!pip install transformers
!pip install datasets

In [4]:
from datasets import load_dataset,load_metric,DatasetDict
from transformers import AutoTokenizer,TFAutoModelForSequenceClassification
import tensorflow as tf
import numpy as np


In [5]:
DATA_PATH = "data/labeled_data.csv"


dataset = load_dataset('csv', data_files=DATA_PATH,split='train')
train_testvalid = dataset.train_test_split()
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split()
# gather everyone if you want to have a single DatasetDict
train_test_valid_dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

Using custom data configuration default-a3092c6d9a4e42e6


Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a3092c6d9a4e42e6/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [6]:
#Remove columns
dataset = train_test_valid_dataset.remove_columns(['hate_speech', 'offensive_language', 'neither','Unnamed: 0', 'count'])


In [7]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
def tokenize_function(examples):
    return tokenizer(examples["tweet"], padding="max_length", truncation=True)



tokenized_datasets = dataset.map(tokenize_function, batched=True)
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["valid"]
test_dataset = tokenized_datasets['test']

In [8]:
#Generate datasets
model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=3)


tf_train_dataset = train_dataset.remove_columns(["tweet"]).with_format("tensorflow")
tf_eval_dataset = eval_dataset.remove_columns(["tweet"]).with_format("tensorflow")
tf_test_dataset = test_dataset.remove_columns(["tweet"]).with_format("tensorflow")

train_features = {x: tf_train_dataset[x].to_tensor() for x in tokenizer.model_input_names}
train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, tf_train_dataset["class"]))
train_tf_dataset = train_tf_dataset.shuffle(len(tf_train_dataset)).batch(8)

eval_features = {x: tf_eval_dataset[x].to_tensor() for x in tokenizer.model_input_names}
eval_tf_dataset = tf.data.Dataset.from_tensor_slices((eval_features, tf_eval_dataset["class"]))
eval_tf_dataset = eval_tf_dataset.batch(8)



test_features = {x: tf_test_dataset[x].to_tensor() for x in tokenizer.model_input_names}
test_tf_dataset = tf.data.Dataset.from_tensor_slices((test_features, tf_test_dataset["class"]))
test_tf_dataset =test_tf_dataset.batch(8)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy(),
)

model.fit(train_tf_dataset, validation_data=eval_tf_dataset, epochs=2)

Epoch 1/2
2324/2324 [==============================] - 2322s 993ms/step - loss: 0.2763 - sparse_categorical_accuracy: 0.9074 - val_loss: 0.2725 - val_sparse_categorical_accuracy: 0.9062
Epoch 2/2
2324/2324 [==============================] - 2305s 992ms/step - loss: 0.2513 - sparse_categorical_accuracy: 0.9131 - val_loss: 0.2929 - val_sparse_categorical_accuracy: 0.9073


In [32]:
#Evaluate on test set
test_loss, test_acc = model.evaluate(test_tf_dataset,verbose=2)

print('\nTest accuracy:', test_acc)

194/194 - 61s - loss: 0.2716 - sparse_categorical_accuracy: 0.9128

Test accuracy: 0.9128469824790955


In [48]:

model.save_pretrained("web-app/hate-speech-tranformers")

